# Make your requests faster

When you start scraping web-pages or requesting APIs, you will be facing a problem when doing a lot of requests: this is really slow!

It's because python is slow! You could say, well it should not be. Let's see how we can speed things up!

## Measure performances

In this notebook we will need to track how much time some code is taking to execute.
To make things easier, we will create a simple decorator that will print the number of micro-seconds a function takes to execute.

A good opportonity to practice decorators in a practical example!

*Note that you need python3.3 or higher.*

In [21]:
import time

def print_timing(func):
    '''Create a timing decorator function use @print_timing just above the function you want to time.'''

    def wrapper(*arg):
        start = time.perf_counter()
        
        # Run the function decorated
        result = func(*arg)

        end = time.perf_counter()
        execution_time = round((end - start), 2)
        print(f'{func.__name__} took {execution_time} sec')
        return result

    return wrapper


@print_timing
def example():
    time.sleep(2)


example()

example took 2.0 sec


## The API

For this example, we will use the [quotable.io](https://api.quotable.io) API. It's an online API you can use to generate random quote.

But feel free to replace `api_url` value with any API you'd like.

In [32]:
api_url = "https://api.quotable.io/random"

## The "classic" way

If you start playing with requests, your should probably have something like this:

In [35]:
import requests


def basic_request(url: str):
    response = requests.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def basic_loop_request(url: str):
    # Query 50 times the API
    for _ in range(50):
        basic_request(url)


basic_loop_request(api_url)

Six essential qualities that are the key to success: Sincerity, personal integrity, humility, courtesy, wisdom, charity.
Always seek out the seed of triumph in every adversity.
Kind words do not cost much. Yet they accomplish much.
Truth, and goodness, and beauty are but different faces of the same all.
However rare true love may be, it is less so than true friendship.
In skating over thin ice our safety is in our speed.
Never give up, for that is just the place and time that the tide will turn.
Don't leave a stone unturned. It's always something, to know you have done the most you could.
This is the final test of a gentleman: his respect for those who can be of no possible value to him.
The greatest obstacle to discovery is not ignorance - it is the illusion of knowledge.
I hear and I forget. I see and I remember. I do and I understand.
Do you want to know who you are? Don't ask. Act! Action will delineate and define you.
A true friend freely, advises justly, assists readily, adventur

### Results

On my machine it took **17.06 sec for 50 requests**. 

Pretty slow right? But why is that?

Each time you make a request, your computer needs to create a new "session", format your request, send it and wait to receive the response before doing it again with the next request.

## The "session" way

To speed this, we can use a **"session"** that will be share by all the requests.

You can picture it as a postman that knows you already, so he knows which bell to ring, where is the mailbox,... Instead of having to search for those each time.

In [37]:
from requests import Session


def session_request(url: str, session: Session):
    # Instead of using request.get, we use our session
    response = session.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def session_loop_request(url: str):
    # Create shared session for all of your requests
    with Session() as session:
        # Query 50 times the API
        for _ in range(50):
            session_request(url, session)


session_loop_request(api_url)

Love is never lost. If not reciprocated, it will flow back and soften and purify the heart.
The conditions of conquest are always easy. We have but to toil awhile, endure awhile, believe always, and never turn back.
The world has the habit of making room for the man whose actions show that he knows where he is going.
Meaning is not what you start with but what you end up with.
One fails forward toward success.
What is not started today is never finished tomorrow.
Science investigates; religion interprets. Science gives man knowledge which is power; religion gives man wisdom which is control.
Success in business requires training and discipline and hard work. But if you're not frightened by these things, the opportunities are just as great today as they ever were.
I love my past. I love my present. I'm not ashamed of what I've had, and I'm not sad because I have it no longer.
Some people are born on third base and go through life thinking they hit a triple.
If the stars should appear bu

### Results

It took me **5.99 sec for 50 requests**. That's better!

And as you can see, I didn't change that much in the code.

## The "Async" way

If you need even more performances, you will need to use [AsyncIo](https://docs.python.org/3/library/asyncio.html).

This is a library to allow you to run asynchronous code.

Why is that more efficiant? Well, when you send a request you need to wait for the response. And during the waiting time, our computer does nothing.
If you count all the time the computer is just "waiting" on 50 or more requests, you will be surprised to see that most of the computing time is just waiting for the server to respond.

[AsyncIo](https://docs.python.org/3/library/asyncio.html) allow you to bypass that.

But as always, it has a cost: complexity.

Making your code async will complixify the code a lot and make the debugging not a pleasant experience. Also, you will go so fast that you could be banned by the server.

My advice? Use it only if you need it.

I will show you a simple example but you want to understand it better, I really advice you **[this video](https://www.youtube.com/watch?v=qAh5dDODJ5k)**!

### Requirements
In order to simplify a bit the code, I will use [httpx](https://www.python-httpx.org/) a python library that is working the same way as the `requests` module but with few helpers for async.

In [42]:
!pip install httpx

### Warning!
This code won't work in jupyter notebook, there are subtilities for async in jupyter notebook. See [this thread](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook) for more informations.

To make it simpler, I will put this code in a .py file and run it in command line:

```python
from httpx import AsyncClient
import asyncio
import time


api_url = "https://api.quotable.io/random"


async def session_request_async(url: str, session: AsyncClient):
    # Instead of using request.get, we use our session
    response = await session.get(url)
    response_json = response.json()
    print(response_json["content"])
    return response_json


async def session_loop_request_async(url: str):
    # Create shared session for all of your requests
    async with AsyncClient() as session:
        # Create a list of empty tasks
        tasks = []
        # Query 50 times the API
        for _ in range(50):
            # Add a request to tasks
            tasks.append(
                asyncio.create_task(
                    session_request_async(url, session)        
                )
            )
        # Now that all the tasks are registred, run them
        responses = await asyncio.gather(*tasks)
            
            


start = time.perf_counter()

# We need to use asyncio.run to run the async function
asyncio.run(session_loop_request_async(api_url))

end = time.perf_counter()
execution_time = round((end - start), 2)
print(f'session_loop_request_async took {execution_time} sec')
```

In [56]:
!python3 ./assets/async_requests.py

Don't cry because it's over. Smile because it happened.
Our kindness may be the most persuasive argument for that which we believe.
The pessimist complains about the wind; the optimist expects it to change; the realist adjusts the sails.
There are two ways to slide easily through life: to believe everything or to doubt everything; both ways save us from thinking.
It is a common experience that a problem difficult at night is resolved in the morning after the committee of sleep has worked on it.
Optimism is the faith that leads to achievement. Nothing can be done without hope and confidence.
Obstacles are those frightful things you see when you take your eyes off your goal.
Irony is the gaiety of reflection and the joy of wisdom.
Twenty years from now you will be more disappointed by the things that you didn't do than by the ones you did do.
An idea that is developed and put into action is more important than an idea that exists only as an idea.
The best and most beautiful things in the

### Results
It only took me **0.8 sec for 50 requests**! That's impressive.

But as you can see, it is harder to write, structure and debug. So make sure you **really** need it if you consider using this method.

## Summary

If we gather all our results:

| Method                     | Execution time for 50 requests |
|----------------------------|--------------------------------|
| `requests.get` loop        | 17.06 sec                  |
| `requests` with `Session`  | 5.99 sec                   |
| `httpx` with `AsyncClient` | 0.8 sec                   |